In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# finebert fine-tuned on 2000 annotated sentences from the ESG domain
# four categories: environment, social, governance, and non-ESG
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg-9-categories',num_labels=9)
tokenizer = BertTokenizer.from_pretrained(
    'yiyanghkust/finbert-esg-9-categories',
)

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
finbert.to(device)
class_labels = ['Climate Change', 'Natural Capital', 'Pollution & Waste',
                'Human Capital', 'Product Liability', 'Community Relations',
                'Corporate Governance', 'Business Ethics & Values', 'Non-ESG']

sentence = 'For 2002, our total net emissions were approximately 60 million metric tons of CO2 equivalents for all businesses and operations we have ﬁnancial interests in, based on its equity share in those businesses and operations.'
inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)

c:\Users\danto\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
inputs.to(device)

outputs = finbert(**inputs)
print(outputs.logits)
# put logits through softmax to get probabilities
print(torch.nn.functional.softmax(outputs.logits, dim=1).detach().cpu().numpy().flatten())
class_predictions = torch.argmax(outputs.logits, dim=1)
print(class_labels[class_predictions])

tensor([[ 6.9215, -0.4468,  0.0691, -1.1357, -0.9401, -1.2870, -0.0733, -1.2235,
         -0.5228]], device='cuda:0', grad_fn=<AddmmBackward0>)
[9.9556559e-01 6.2816904e-04 1.0522981e-03 3.1543022e-04 3.8357917e-04
 2.7113734e-04 9.1263442e-04 2.8890101e-04 5.8221794e-04]
Climate Change
